In [ ]:
'''Import all files:
- Embeddings of the front pages ('better_cleaned_embeddings.csv')
- Embeddings of the articles ('articles_with_embeddings')
- NLI data ('nli_analysis_all_articles.csv')
- Cosine ('cosine_features_per_article.csv')
- Data for each stock ('SPY.csv', 'VT.csv', etc)
'''

from google.colab import files
uploaded = files.upload()

Saving cosine_features_per_article.csv to cosine_features_per_article.csv


In [ ]:
#Installs
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
#Imports
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow.keras.optimizers
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import keras_tuner as kt
import os
import tensorflow.keras.backend as K
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
#Neural network for embeddings on front page
#Regression

#Import embeddings, convert to matrix
embeddings = pd.read_csv("better_cleaned_embeddings.csv")
embeddings['Embedding'] = embeddings['Embedding'].apply(ast.literal_eval)
embedding_matrix = np.vstack(embeddings['Embedding'].values)

X = np.array(embedding_matrix).astype('float32')
y = np.array(embeddings['SPY'].to_numpy()).astype('float32').reshape(-1, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize inputs
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  #Linear activation function

# Compile and train
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val_scaled, y_val)

# Predictions on test set
y_pred = model.predict(X_test_scaled)

# Save predictions alongside actuals
results = pd.DataFrame({
    'predicted_change': y_pred.flatten(),
    'actual_change': y_test.flatten()
})

print(results.head())

r2 = r2_score(results['actual_change'], results['predicted_change'])
print("R²:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 3.5123 - mae: 1.2182 - val_loss: 1.0362 - val_mae: 0.6803
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.3605 - mae: 0.7530 - val_loss: 1.0400 - val_mae: 0.6862
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.2407 - mae: 0.7122 - val_loss: 1.1031 - val_mae: 0.7115
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.1597 - mae: 0.7075 - val_loss: 1.0908 - val_mae: 0.7095
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.1138 - mae: 0.6989 - val_loss: 1.1142 - val_mae: 0.7203
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.9058 - mae: 0.6714 - val_loss: 1.2327 - val_mae: 0.7653
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 1.4509 - mae: 0.7314 - val_loss: 1.0560 - val_mae: 0.6900
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 1.1053 - mae: 0.6884 - val_loss: 1.0770 - val_mae: 0.6967
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.3786 -

In [ ]:
#Neural network for embeddings on the full front pages
#classification

X = np.array(embedding_matrix).astype('float32')
#binary output
y = (embeddings['SPY'].to_numpy() > 0).astype('int32').reshape(-1, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Binary output

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val)
print("Validation Accuracy:", val_accuracy)

y_pred_probs = model.predict(X_test_scaled)
y_pred_labels = (y_pred_probs > 0.5).astype('int32')

results = pd.DataFrame({
    'predicted_label': y_pred_labels.flatten(),
    'actual_label': y_test.flatten()
})

print(results.head())

print(classification_report(y_test, y_pred_labels))

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5264 - loss: 0.8283 - val_accuracy: 0.5496 - val_loss: 0.6927
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5874 - loss: 0.6847 - val_accuracy: 0.5305 - val_loss: 0.7001
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6288 - loss: 0.6498 - val_accuracy: 0.4695 - val_loss: 0.7398
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6725 - loss: 0.6122 - val_accuracy: 0.5534 - val_loss: 0.6906
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6914 - loss: 0.5730 - val_accuracy: 0.5573 - val_loss: 0.7593
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7294 - loss: 0.5175 - val_accuracy: 0.5115 - val_loss: 0.8098
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7493 - loss: 0.4730 - val_accuracy: 0.5496 - val_loss: 0.7752
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7550 - loss: 0.4649 - val_accuracy: 0.5611 - val_l

In [ ]:
#Hyperparameter tuning for embeddings on the full front pages
#regression

y_regression = y.astype('float32')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_scaled, X_temp, y_train, y_temp = train_test_split(X_scaled, y_regression, test_size=0.2, random_state=42)
X_val_scaled, X_test_scaled, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

def build_model(hp):
    model = Sequential()
    model.add(Dense(
        hp.Int('units_input', min_value=64, max_value=256, step=64),
        activation=hp.Choice('activation_input', ['relu', 'tanh']),
        input_shape=(X.shape[1],)
    ))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh'])
        ))
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(rate=hp.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='linear'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')),
        loss='mean_squared_error',
        metrics=['mae']
    )

    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_mae',
    max_epochs=30,
    factor=3,
    directory='hyperband_dir',
    project_name='tlt_direction_regression'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(
    X_train_scaled, y_train,
    epochs=50,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping],
    batch_size=64
)

best_models = tuner.get_best_models(num_models=15)

for idx, model in enumerate(best_models):
    test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
    print(f"Model {idx+1}: Test MAE = {test_mae:.4f}, Loss (MSE) = {test_loss:.4f}")

Trial 90 Complete [00h 00m 25s]
val_mae: 0.5124397277832031

Best val_mae So Far: 0.4393603801727295
Total elapsed time: 00h 14m 51s
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3513 - mae: 0.5221 
Model 1: Test MAE = 0.5267, Loss (MSE) = 0.3536
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2900 - mae: 0.4888  
Model 2: Test MAE = 0.4975, Loss (MSE) = 0.3019
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2985 - mae: 0.4885  
Model 3: Test MAE = 0.4930, Loss (MSE) = 0.3024
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3238 - mae: 0.5223  
Model 4: Test MAE = 0.5069, Loss (MSE) = 0.3077
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3069 - mae: 0.5067  
Model 5: Test MAE = 0.5014, Loss (MSE) = 0.3024
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3113 - mae: 0.5091  
Model 6: Test MAE = 0.4961, Loss (MSE) = 0.2976
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3238 - mae: 0.5170  
Model 7: Test MAE = 0.5086, Loss (MSE) = 0.3146
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3152 - 

In [ ]:
for idx, model in enumerate(best_models):
    y_pred = model.predict(X_test_scaled).flatten()
    r2 = r2_score(y_test, y_pred)
    print(f"Model {idx+1}: R² = {r2:.4f}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
Model 1: R² = -0.4516
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
Model 2: R² = -0.2393
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Model 3: R² = -0.2415
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Model 4: R² = -0.2631
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Model 5: R² = -0.2414
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Model 6: R² = -0.2216
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Model 7: R² = -0.2916
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Model 8: R² = -0.3231
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Model 9: R² = -0.2228
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Model 10: R² = -0.3272
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Model 11: R² = -0.2807
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Model 12: R² = -0.1154
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Model 13: R² = -0.6765
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Model 14: R² = -0.5682
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Model 15: R² = -0.2642


In [ ]:
#Hyperparameter tuning for embeddings on the full front pages
#classification

y_binary = (y > 0).astype('int32')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_scaled, X_temp, y_train, y_temp = train_test_split(X_scaled, y_binary, test_size=0.2, random_state=42)
X_val_scaled, X_test_scaled, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

def build_model(hp):
    model = Sequential()
    model.add(Dense(
        hp.Int('units_input', min_value=64, max_value=256, step=64),
        activation=hp.Choice('activation_input', ['relu', 'tanh']),
        input_shape=(X.shape[1],)
    ))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh'])
        ))
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(rate=hp.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    factor=3,
    directory='hyperband_dir',
    project_name='tlt_direction_classification'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(
    X_train_scaled, y_train,
    epochs=50,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping],
    batch_size=64
)

best_models = tuner.get_best_models(num_models=15)

for idx, model in enumerate(best_models):
    y_pred_probs = model.predict(X_test_scaled)
    y_pred = (y_pred_probs > 0.5).astype('int32')

    acc = accuracy_score(y_test, y_pred)
    print(f"Model {idx+1}: Accuracy = {acc:.4f}")
    print(classification_report(y_test, y_pred))

Trial 90 Complete [00h 00m 13s]
val_accuracy: 0.5419847369194031

Best val_accuracy So Far: 0.6030534505844116
Total elapsed time: 00h 15m 44s
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Model 1: Accuracy = 0.5038
              precision    recall  f1-score   support

           0       0.40      0.37      0.39       110
           1       0.57      0.60      0.58       152

    accuracy                           0.50       262
   macro avg       0.49      0.49      0.49       262
weighted avg       0.50      0.50      0.50       262

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Model 2: Accuracy = 0.5229
              precision    recall  f1-score   support

           0       0.42      0.38      0.40       110
           1       0.58      0.62      0.60       152

    accuracy                           0.52       262
   macro avg       0.50      0.50      0.50       262
weighted avg       0.52      0.52      0.52       262

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Model 3: Accuracy = 0.5420
        

In [ ]:
#Neural network for embeddings on the articles
#regression

split_articles = pd.read_csv("articles_with_embeddings.csv")
split_articles['Date'] = pd.to_datetime(split_articles['Date']).dt.date

tickers = ["SPY", "QQQ", "DIA", "TLT", "IEF", "VT", "IWM"]

for ticker in tickers:
    df_stock = pd.read_csv(f"{ticker}.csv")

    df_stock['timestamp'] = pd.to_datetime(df_stock['timestamp']) + pd.Timedelta(days=-1)
    df_stock['timestamp'] = df_stock['timestamp'].dt.date

    if df_stock['timestamp'].max() < min(split_articles['Date']):
        print(f"Skipping {ticker} — no overlapping dates.")
        continue

    df_change = df_stock[['timestamp', 'prev_day_percent_change']].rename(columns={
        'timestamp': 'Date',
        'prev_day_percent_change': ticker
    })
    split_articles = pd.merge(split_articles, df_change, on='Date', how='left')

split_articles = split_articles.dropna(subset=tickers, how='all')

split_articles['Embedding'] = split_articles['Embedding'].apply(ast.literal_eval)
embedding_matrix = np.vstack(split_articles['Embedding'].values)

X = np.array(embedding_matrix).astype('float32')
y = np.array(split_articles['SPY'].to_numpy()).astype('float32').reshape(-1, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

val_loss, val_mae = model.evaluate(X_val_scaled, y_val)

y_pred = model.predict(X_test_scaled)

results = pd.DataFrame({
    'predicted_change': y_pred.flatten(),
    'actual_change': y_test.flatten()
})

print(results.head())

r2 = r2_score(results['actual_change'], results['predicted_change'])
print("R²:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 2.8843 - mae: 1.1075 - val_loss: 1.1905 - val_mae: 0.7197
Epoch 2/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.5175 - mae: 0.7920 - val_loss: 1.2131 - val_mae: 0.7209
Epoch 3/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.4605 - mae: 0.7852 - val_loss: 1.2289 - val_mae: 0.7321
Epoch 4/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.4897 - mae: 0.7828 - val_loss: 1.2210 - val_mae: 0.7286
Epoch 5/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.6125 - mae: 0.7863 - val_loss: 1.2678 - val_mae: 0.7377
Epoch 6/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.5217 - mae: 0.7758 - val_loss: 1.2001 - val_mae: 0.7232
Epoch 7/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.2523 - mae: 0.7607 - val_loss: 1.5394 - val_mae: 0.7674
Epoch 8/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.3568 - mae: 0.7443 - val_loss: 1.2208 - val_mae: 0.7270
Epoch 9/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss

In [ ]:
#Neural network for embeddings on the articles
#classification

X = np.array(embedding_matrix).astype('float32')
#binary output
y = (split_articles['SPY'].to_numpy() > 0).astype('int32').reshape(-1, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Binary output

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val)
print("Validation Accuracy:", val_accuracy)

y_pred_probs = model.predict(X_test_scaled)
y_pred_labels = (y_pred_probs > 0.5).astype('int32')

results = pd.DataFrame({
    'predicted_label': y_pred_labels.flatten(),
    'actual_label': y_test.flatten()
})

print(results.head())

print(classification_report(y_test, y_pred_labels))

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5103 - loss: 0.8296 - val_accuracy: 0.5368 - val_loss: 0.6971
Epoch 2/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5377 - loss: 0.6918 - val_accuracy: 0.5368 - val_loss: 0.6887
Epoch 3/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5494 - loss: 0.6857 - val_accuracy: 0.5629 - val_loss: 0.6963
Epoch 4/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5788 - loss: 0.6796 - val_accuracy: 0.4656 - val_loss: 0.7167
Epoch 5/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5766 - loss: 0.6772 - val_accuracy: 0.5487 - val_loss: 0.7448
Epoch 6/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5904 - loss: 0.6501 - val_accuracy: 0.5416 - val_loss: 0.7496
Epoch 7/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5962 - loss: 0.6403 - val_accuracy: 0.5297 - val_loss: 0.7863
Epoch 8/150
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6300 - loss: 0.6107 - val_accuracy: 0.5226 - val_lo

In [ ]:
#Neural network for cosine/MNLI
#regression

cosine = pd.read_csv("cosine_features_per_article.csv")
nli = pd.read_csv("nli_analysis_all_articles.csv")

merged = pd.merge(cosine, nli, on="Article_Index")
spy = pd.read_csv("SPY.csv")
vt = pd.read_csv("VT.csv")
spy['timestamp'] = pd.to_datetime(spy['timestamp'])
merged['Date'] = pd.to_datetime(merged['Date'])
merged_with_market = pd.merge(
    merged,
    spy[['timestamp', 'prev_day_percent_change']],
    left_on='Date',
    right_on='timestamp',
    how='inner'
)
merged_with_market = merged_with_market.drop(columns=['timestamp'])
merged_with_market.rename(columns={'prev_day_percent_change': 'market_change'}, inplace=True)
merged_with_market.head()
drop_cols = ['Article_Index', 'Date', 'Article_Text'] + \
            [col for col in merged_with_market.columns if 'Sentence' in col or 'Error' in col]
X = merged_with_market.drop(columns=drop_cols)
X = X.drop(columns='market_change')
y = merged_with_market['market_change']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))

model.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['mae']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

val_loss, val_mae = model.evaluate(X_val_scaled, y_val)

y_pred = model.predict(X_test_scaled)

results = pd.DataFrame({
    'predicted_change': y_pred.flatten(),
    'actual_change': np.array(y_test).flatten()
})

print(results.head())

r2 = r2_score(results['actual_change'], results['predicted_change'])
print("R²:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 2.1354 - mae: 0.9556 - val_loss: 1.3574 - val_mae: 0.7522
Epoch 2/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.5984 - mae: 0.8076 - val_loss: 1.3616 - val_mae: 0.7544
Epoch 3/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.6339 - mae: 0.8110 - val_loss: 1.3542 - val_mae: 0.7548
Epoch 4/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.4322 - mae: 0.7763 - val_loss: 1.3599 - val_mae: 0.7543
Epoch 5/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.5084 - mae: 0.7794 - val_loss: 1.3315 - val_mae: 0.7544
Epoch 6/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.7887 - mae: 0.8035 - val_loss: 1.3634 - val_mae: 0.7543
Epoch 7/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.7537 - mae: 0.8201 - val_loss: 1.3643 - val_mae: 0.7557
Epoch 8/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.6711 - mae: 0.8093 - val_loss: 1.3457 - val_mae: 0.7657
Epoch 9/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.4785 -

In [ ]:
#Neural network for cosine/MNLI
#classification

y = (merged_with_market['market_change'].to_numpy() > 0).astype('int32').reshape(-1, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val)
print("Validation Accuracy:", val_accuracy)

y_pred_probs = model.predict(X_test_scaled)
y_pred_labels = (y_pred_probs > 0.5).astype('int32')

results = pd.DataFrame({
    'predicted_label': y_pred_labels.flatten(),
    'actual_label': y_test.flatten()
})

print(results.head())

print(classification_report(y_test, y_pred_labels))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5242 - loss: 0.7172 - val_accuracy: 0.5469 - val_loss: 0.6889
Epoch 2/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5618 - loss: 0.6867 - val_accuracy: 0.5423 - val_loss: 0.6889
Epoch 3/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5687 - loss: 0.6869 - val_accuracy: 0.5355 - val_loss: 0.6896
Epoch 4/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5576 - loss: 0.6869 - val_accuracy: 0.5469 - val_loss: 0.6923
Epoch 5/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5699 - loss: 0.6897 - val_accuracy: 0.5652 - val_loss: 0.6869
Epoch 6/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5562 - loss: 0.6862 - val_accuracy: 0.5492 - val_loss: 0.6895
Epoch 7/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5734 - loss: 0.6825 - val_accuracy: 0.5446 - val_loss: 0.6883
Epoch 8/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5718 - loss: 0.6848 - val_accuracy: 0.5469

In [ ]:
#Hyperparameter tuning for cosine/MNLI
#regression
y_regression = y.astype('float32')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_scaled, X_temp, y_train, y_temp = train_test_split(X_scaled, y_regression, test_size=0.2, random_state=42)
X_val_scaled, X_test_scaled, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

def build_model(hp):
    model = Sequential()
    model.add(Dense(
        hp.Int('units_input', min_value=64, max_value=256, step=64),
        activation=hp.Choice('activation_input', ['relu', 'tanh']),
        input_shape=(X.shape[1],)
    ))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh'])
        ))
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(rate=hp.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='linear'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')),
        loss='mean_squared_error',
        metrics=['mae']
    )

    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_mae',
    max_epochs=30,
    factor=3,
    directory='hyperband_dir',
    project_name='tlt_direction_regression'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(
    X_train_scaled, y_train,
    epochs=50,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping],
    batch_size=64
)

best_models = tuner.get_best_models(num_models=15)

for idx, model in enumerate(best_models):
    test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
    print(f"Model {idx+1}: Test MAE = {test_mae:.4f}, Loss (MSE) = {test_loss:.4f}")
    y_pred = model.predict(X_test_scaled).flatten()
    r2 = r2_score(y_test, y_pred)
    print(f"Model {idx+1}: R² = {r2:.4f}")

Trial 90 Complete [00h 00m 19s]
val_mae: 0.49293482303619385

Best val_mae So Far: 0.484881728887558
Total elapsed time: 00h 21m 34s
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2493 - mae: 0.4679  
Model 1: Test MAE = 0.4770, Loss (MSE) = 0.2586
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Model 1: R² = -0.0477
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2617 - mae: 0.4864
Model 2: Test MAE = 0.4930, Loss (MSE) = 0.2696
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Model 2: R² = -0.0922
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2479 - mae: 0.4733
Model 3: Test MAE = 0.4795, Loss (MSE) = 0.2532
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Model 3: R² = -0.0258
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.2469 - mae: 0.4732
Model 4: Test MAE = 0.4798, Loss (MSE) = 0.2534
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Model 4: R² = -0.0264
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2526 - mae: 0.4728  
Model 5: Test MAE = 0.4805, Loss (MSE) = 0.2604
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/st

In [ ]:
#Hyperparameter tuning for cosine/MNLI
#classification

def binary_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        alpha_t = tf.where(tf.equal(y_true, 1), alpha, 1 - alpha)
        focal_loss = -alpha_t * K.pow(1. - p_t, gamma) * K.log(p_t)
        return K.mean(focal_loss)
    return loss


y_binary = (y > 0).astype(int)
y_train_flat = y_train.ravel()  # or use .flatten()


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_scaled, X_temp, y_train, y_temp = train_test_split(X_scaled, y_binary, test_size=0.2, random_state=42)
X_val_scaled, X_test_scaled, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_flat), y=y_train_flat)
class_weight_dict = dict(zip(np.unique(y_train_flat), class_weights))

def build_model(hp):
    model = Sequential()
    model.add(Dense(
        hp.Int('units_input', min_value=64, max_value=256, step=64),
        activation=hp.Choice('activation_input', ['relu', 'tanh']),
        input_shape=(X.shape[1],)
    ))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh'])
        ))
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(rate=hp.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')),
    loss=binary_focal_loss(gamma=2.0, alpha=0.25),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
    )


    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    factor=3,
    directory='hyperband_dir',
    project_name='tlt_direction_classification_focal'
)

tuner.search(
    X_train_scaled, y_train,
    epochs=50,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping],
    batch_size=64,
    class_weight=class_weight_dict
)

best_models = tuner.get_best_models(num_models=15)

for idx, model in enumerate(best_models):
    y_pred_probs = model.predict(X_test_scaled)
    y_pred = (y_pred_probs > 0.5).astype('int32')

    acc = accuracy_score(y_test, y_pred)
    print(f"Model {idx+1}: Accuracy = {acc:.4f}")
    print(classification_report(y_test, y_pred))




Model 1 Evaluation:
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
              precision    recall  f1-score   support

           0     0.4286    0.0773    0.1310       194
           1     0.5547    0.9177    0.6915       243

    accuracy                         0.5446       437
   macro avg     0.4916    0.4975    0.4112       437
weighted avg     0.4987    0.5446    0.4427       437


Model 2 Evaluation:
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
              precision    recall  f1-score   support

           0     0.4524    0.0979    0.1610       194
           1     0.5570    0.9053    0.6897       243

    accuracy                         0.5469       437
   macro avg     0.5047    0.5016    0.4253       437
weighted avg     0.5105    0.5469    0.4550       437


Model 3 Evaluation:
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
              precision    recall  f1-score   support

           0     0.4634    0.0979    0.1617       194
           1     0.5581    0.9095    0.6917       243

 

In [ ]:
#Example trading bot

money = 10000

for index, row in results.iterrows():
    prediction = row['predicted_change']
    bet = prediction * money / 10
    result = bet * (row['actual_change'] / 100.0)
    money += result

    print(f"prediction: {prediction:.2f}, real: {row['actual_change']:.2f}%, result: {result:.3f}, money: {money:.3f}")


prediction: 0.10, real: 0.36%, result: 0.376, money: 10000.376
prediction: 0.05, real: 0.07%, result: 0.035, money: 10000.411
prediction: 0.05, real: 0.45%, result: 0.201, money: 10000.612
prediction: 0.05, real: 0.77%, result: 0.347, money: 10000.960
prediction: 0.05, real: -1.92%, result: -0.865, money: 10000.095
prediction: 0.05, real: 1.16%, result: 0.521, money: 10000.616
prediction: 0.06, real: 0.09%, result: 0.054, money: 10000.670
prediction: -0.08, real: 0.57%, result: -0.447, money: 10000.223
prediction: -0.03, real: 0.13%, result: -0.037, money: 10000.186
prediction: 0.05, real: -2.44%, result: -1.267, money: 9998.919
prediction: 0.05, real: 0.05%, result: 0.023, money: 9998.942
prediction: 0.02, real: -0.97%, result: -0.200, money: 9998.742
prediction: 0.06, real: -0.73%, result: -0.470, money: 9998.272
prediction: 0.10, real: 0.62%, result: 0.645, money: 9998.917
prediction: 0.07, real: -0.73%, result: -0.502, money: 9998.415
prediction: -0.69, real: 0.86%, result: -5.975,